App validation dataset (there are sutle changes compared with the previous this mainly comes since we rule out a few calculations)

In [1]:
import math
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import distance
import utm

from GPSPhoto import gpsphoto


In [2]:
def get_distance_between_two_consecutive_images(cordinate_1,cordinate_2):
    return distance.euclidean(cordinate_1,cordinate_2)
def distance_two_points_along_x(A,B):
    return A[0]-B[0]
def distance_two_points_along_y(A,B):
    return A[1]-B[1]

In [3]:
def clear_distortions(img_before_distance):
    
    #mi8 (change)
    mtx=[[3095.73828160516,0,2016.7322805317165],[0,3102.1298848836359,1510.5872677598889],[0,0,1]] 
    dist=[ 0.21522014938761402 , -0.29536385974743162 , -0.0062158816142389856,0.0031376495475325976 ]
    
    #converting into numpy
    mtx = np.array(mtx)
    dist=np.array(dist)
    
    #image dimensions
    image_height,image_width,_=img_before_distance.shape

    #pumping distortion matrix
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(image_width,image_height),1,(image_width,image_height))

    #undistort image_before_distance
    img_before_distance = cv2.undistort(img_before_distance, mtx, dist, None, newcameramtx)
    x,y,w,h = roi
    undistorted_image = img_before_distance[y:y+h, x:x+w]

    #undistort image_after_distance

    return undistorted_image


In [4]:
def parsing_annotations(highway_sign_annotations,image_file_name):
   
    highway_signs = pd.read_csv(highway_sign_annotations)
    
    for index,row in highway_signs.iterrows():
        if row['frame_number']== image_file_name:
            sign_top_left_x=row['top_x']
            sign_top_left_y=row['top_y']
            sign_width=row['width']
            sign_height=row['height']
            class_of_sign=['class']

    return [sign_top_left_x,sign_top_left_y,sign_width,sign_height,class_of_sign]


In [5]:
def find_center_of_sign(sign_details_list):
    sign_top_left_x=sign_details_list[0]
    sign_top_left_y=sign_details_list[1]
    sign_width=sign_details_list[2]
    sign_height=sign_details_list[3]

    location_sign=(int((sign_top_left_x+sign_top_left_x+sign_width)/2),int((sign_top_left_y+sign_top_left_y+sign_height)/2))
    print("center of the sign",location_sign)
    print("------------------------------------------")
    return location_sign

In [6]:
def finding_relative_location_of_image(center_sign,image_width):
    if center_sign[0]<image_width/2:
        print("sign is to the left of the vehicle")
        return -1
    elif center_sign[0]>image_width/2:
        print("sign is to the right of the vehicle")
        return 1
    else:
        print("sign is alligned with the optical axis")
        return 0


In [22]:
def trignometric_calculations(x1,x2,f):
    dst= 3.16992
    print('The images are taken at a distance of {} m '.format(dst)) 
    if(x2-x1)!=0:
        l =  dst * x1/(x2-x1) 
        w = l * (x2)/f 
    elif (x2-x1)==0:
        l=dst * x1
        w=l*(x2)/f
    #w--> how right or how left the sign is (x-axis)
    #l--> how ahead the sign is (y-axis)
    print('how inclined:', w) #add to the x-cordinate
    print('how ahead:', l) #add to the y-cordinate
    

In [23]:
def calculation_of_distances(image_file_name_before_distance,image_file_name_after_distance,sign_annotations,f):
    #load image     
    #print(image_file_name_before_distance,image_file_name_after_distance)
    img_before_distance = cv2.imread(image_file_name_before_distance)
    img_after_distance  = cv2.imread(image_file_name_after_distance)
    #clear distortions
    img_before_distance = clear_distortions(img_before_distance)
    img_after_distance  = clear_distortions(img_after_distance)
    #calculate image center and dimensions
    image_height,image_width,_=img_before_distance.shape
    image_center = (int(image_width/2),int(image_height/2))
    print('image_center',image_center)
    print('---------------------------------')
    print("width of image",image_width)
    print("height of image",image_height)
    print("-----------------------------------")
    #parse annotations for details
    sign_before_distance = parsing_annotations(sign_annotations,image_file_name_before_distance)
    #print(sign_before_distance)
    sign_after_distance  = parsing_annotations(sign_annotations,image_file_name_after_distance)
    #print(sign_after_distance)
    #if we are dealing with the same image proceed as else inform and kill 
    if sign_after_distance[4]==sign_before_distance[4]:
        #Find center of sign
        center_before_distance = find_center_of_sign(sign_before_distance)
        center_after_distance  = find_center_of_sign(sign_after_distance)
        #distance between center and sign
        x1=distance_two_points_along_x(center_before_distance,image_center)
        print('distance of the sign from the center image 1',x1)
        x2=distance_two_points_along_x(center_after_distance,image_center)
        print('distance of the sign from the center image 2',x2)
        print("-------------------------------------------------------")
        #getting camera_cordinates_to_calculate distance between images
        #camera_cordinates=parsing_camrea_annotations(image_file_name_after_distance,camera_annotations)
        #camera_cordinates_image_1=parsing_camrea_annotations(image_file_name_before_distance,camera_annotations)
        #getting distances from camera
        distance_tuple=trignometric_calculations(x1,x2,f)
        #understanding spatial location
        right_or_left = finding_relative_location_of_image(center_after_distance,image_width)
        #adding and subtracting images 
        #final_positions = camera_to_sign(camera_cordinates,distance_tuple,right_or_left)
        #error_analysis(final_positions)
        return distance_tuple
    else:
        print('Sorry, We could not find the same sign on both the images')
        return (0,0)


In [24]:
calculation_of_distances('104921.jpg','105857.jpg','sign_annotations.csv',f=3000)

image_center (1953, 1467)
---------------------------------
width of image 3907
height of image 2934
-----------------------------------
center of the sign (2984, 1028)
------------------------------------------
center of the sign (3058, 1020)
------------------------------------------
distance of the sign from the center image 1 1031
distance of the sign from the center image 2 1105
-------------------------------------------------------
The images are taken at a distance of 3.16992 m 
how inclined: 16.26732977297297
how ahead: 44.164696216216214
sign is to the right of the vehicle


In [25]:
calculation_of_distances('105857.jpg','110233.jpg','sign_annotations.csv',f=3000)

image_center (1953, 1467)
---------------------------------
width of image 3907
height of image 2934
-----------------------------------
center of the sign (3058, 1020)
------------------------------------------
center of the sign (3544, 1163)
------------------------------------------
distance of the sign from the center image 1 1105
distance of the sign from the center image 2 1591
-------------------------------------------------------
The images are taken at a distance of 3.16992 m 
how inclined: 3.8222864921810693
how ahead: 7.207328395061728
sign is to the right of the vehicle


In [26]:
calculation_of_distances('110233.jpg','110514.jpg','sign_annotations.csv',f=3000)

image_center (1953, 1467)
---------------------------------
width of image 3907
height of image 2934
-----------------------------------
center of the sign (3544, 1163)
------------------------------------------
center of the sign (3236, 1009)
------------------------------------------
distance of the sign from the center image 1 1591
distance of the sign from the center image 2 1283
-------------------------------------------------------
The images are taken at a distance of 3.16992 m 
how inclined: -7.002823278961038
how ahead: -16.37448935064935
sign is to the right of the vehicle
